In [71]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Model, Sequential
from keras.layers import Conv2D, Dense, Flatten, Input
from keras.layers.merge import Concatenate
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [83]:
data = open("body.txt", "r")
tdata = np.empty((0,48), dtype=float)
tlabel = []
train_images = []

for d in data.readlines():
    tdata = np.array([float(int(t)) for t in d.split(" ")[24:48]]).reshape((1,24))
    tlabel.append(int(d.split(" ")[48].strip("\n")))
    tdata = tdata.reshape(1,6,4)
    max0 = abs(np.max(tdata[0]))

    if (max0 != 0):
        tdata[0] = tdata[0] / max0
    
    train_images.append(tdata)

train_images = np.array(train_images)
tlabel = np.array(to_categorical(tlabel))
print(train_images.shape)
print(tlabel.shape)

(206, 1, 6, 4)
(206, 2)


In [84]:
train_images = np.moveaxis(train_images, [1,2,3], [3,1,2])
print(train_images.shape)

(206, 6, 4, 1)


In [85]:
x_train, x_test, y_train, y_test = train_test_split(train_images, tlabel, test_size=0.05)

In [86]:
num_filters = 1
filter_size = 2
initializer = keras.initializers.RandomNormal(mean=0., stddev=1.)

model = Sequential([
  Conv2D(num_filters, filter_size, input_shape=(6,4,1), kernel_initializer=initializer),
  Conv2D(num_filters, filter_size, input_shape=(5,3), kernel_initializer=initializer),
  Flatten(),
  Dense(16, activation='sigmoid', kernel_initializer=initializer),
  Dense(8, activation='sigmoid', kernel_initializer=initializer),
  Dense(4, activation='sigmoid', kernel_initializer=initializer),
  Dense(2, activation='softmax', kernel_initializer=initializer)
])

In [87]:
opt = keras.optimizers.Nadam(
  learning_rate=0.001,
  beta_1=0.9,
  beta_2=0.999,
  epsilon=1e-07
  )

model.compile(
  loss='binary_crossentropy',
  optimizer=opt,
  metrics=['accuracy'],
)

In [15]:
model.fit(
    x_train,
    y_train,
    epochs=1000,
    validation_data=(x_test, y_test)
    )

Train on 195 samples, validate on 11 samples
Epoch 1/1000
195/195 [==============================] - 3s 14ms/step - loss: 1.1756 - accuracy: 0.2923 - val_loss: 1.2836 - val_accuracy: 0.1818
Epoch 2/1000
195/195 [==============================] - 0s 287us/step - loss: 1.1353 - accuracy: 0.2923 - val_loss: 1.2322 - val_accuracy: 0.1818
Epoch 3/1000
195/195 [==============================] - 0s 287us/step - loss: 1.0920 - accuracy: 0.2923 - val_loss: 1.1817 - val_accuracy: 0.1818
Epoch 4/1000
195/195 [==============================] - 0s 492us/step - loss: 1.0510 - accuracy: 0.2923 - val_loss: 1.1352 - val_accuracy: 0.1818
Epoch 5/1000
195/195 [==============================] - 0s 328us/step - loss: 1.0133 - accuracy: 0.2923 - val_loss: 1.0908 - val_accuracy: 0.1818
Epoch 6/1000
195/195 [==============================] - 0s 287us/step - loss: 0.9775 - accuracy: 0.2923 - val_loss: 1.0481 - val_accuracy: 0.1818
Epoch 7/1000
195/195 [==============================] - 0s 369us/step - loss: 0.

In [16]:
model.save_weights('body.h5')

In [88]:
model.load_weights('body.h5')

In [89]:
weights = np.array(model.get_weights())
print(weights)

[array([[[[3.1293905 ]],

        [[0.38185883]]],


       [[[1.4910437 ]],

        [[0.05645785]]]], dtype=float32)
 array([-0.1497506], dtype=float32)
 array([[[[ 1.6941894 ]],

        [[-0.6454651 ]]],


       [[[-0.74527496]],

        [[-0.9227969 ]]]], dtype=float32)
 array([-0.10511473], dtype=float32)
 array([[ 0.19917633,  0.06279462, -1.3557397 ,  1.3104193 ,  0.8536848 ,
         0.68112606,  2.0083232 ,  0.87816805, -2.7226338 , -1.6500236 ,
        -0.25663653, -0.4320858 , -0.66244084,  1.1394695 ,  0.14960255,
        -3.0444953 ],
       [-2.0696588 , -0.9107198 , -2.7956095 , -1.1170474 ,  0.09452901,
         1.9432744 ,  2.2903168 ,  1.5204856 , -0.4202928 , -0.34525982,
         0.25812423, -0.02076975, -1.4904209 , -1.0935072 ,  0.2568037 ,
        -0.21209562],
       [ 1.7220501 ,  0.48881423,  1.138903  , -0.25011635,  2.2405968 ,
        -0.18214916, -0.9825251 , -1.8588558 , -0.20174764,  2.8235383 ,
         2.8775408 ,  0.21451843,  1.1345357 , -0.603684

In [90]:
weight_file = open("bodyOutput.txt", "w")

for w in weights[0:]:
    weight_file.write("{")

    if(len(w.shape)==2):
        for i in np.array(w):
            weight_file.write("{")
            i=np.array(i)

            for j in np.array(i):
                weight_file.write(str(j) + ", ")
            
            weight_file.write("}, \n")
    
    else:
        for j in np.array(w):
            weight_file.write(str(j) + ", ")

    weight_file.write("}\n\n")

weight_file.flush()
weight_file.close()

In [91]:
testInput = np.array([[[15.0,90.0,11.0,-1.0],[41.0,23.0,22.0,2.0],[27.0,57.0,41.0,0.0],
[22.0,80.0,30.0,0.0],[28.0,12.0,9.0,-1.0],[38.0,8.0,3.0,0.0]]]).reshape(1,1,6,4)

In [92]:
max0 = abs(np.max(testInput[0,0]))

if (max0 != 0):
    testInput[0,0] = testInput[0,0] / max0

In [93]:
testInput = np.moveaxis(testInput, [1,2,3], [3,1,2])
model.predict(testInput)

array([[0.00147385, 0.9985261 ]], dtype=float32)

In [94]:
outputs = [K.function([model.input], [layer.output])([testInput,1]) for layer in model.layers]
print(outputs)

[[array([[[[1.4473547 ],
         [3.421157  ],
         [0.59421957]],

        [[1.8565165 ],
         [1.7133732 ],
         [1.3029506 ]],

        [[1.4455726 ],
         [3.3503458 ],
         [1.7728752 ]],

        [[1.4260496 ],
         [2.9636674 ],
         [1.0418566 ]],

        [[1.5093224 ],
         [0.4401065 ],
         [0.20864703]]]], dtype=float32)], [array([[[[-2.8259697],
         [ 2.8281322]],

        [[-2.2347846],
         [-2.1762779]],

        [[-3.616234 ],
         [ 1.2565076]],

        [[-1.1330494],
         [ 3.7228777]]]], dtype=float32)], [array([[-2.8259697,  2.8281322, -2.2347846, -2.1762779, -3.616234 ,
         1.2565076, -1.1330494,  3.7228777]], dtype=float32)], [array([[3.8743344e-01, 3.3063492e-07, 1.9145139e-07, 7.7010782e-07,
        8.8519082e-05, 9.9745566e-01, 9.9896920e-01, 9.9963892e-01,
        9.9999785e-01, 9.3442190e-01, 6.1406439e-08, 4.1030644e-06,
        1.3121377e-02, 4.0071356e-07, 3.1715558e-08, 9.9878687e-01]],
      d

In [60]:
data = open("weight.txt", "r")
tdata = []
tlabel = np.empty((0,1), dtype=float)
bodyOutput = []
train_images = []

for d in data.readlines():
    temp = np.array([float(t) for t in d.split(" ")[24:49]]).reshape((1,25))
    tdata = temp[0][:24]
    bodyOutput.append(temp[0][24])
    tlabel = np.append(tlabel, np.array([float(d.split(" ")[49].strip("\n"))]).reshape(1,1), axis=0)
    tdata = tdata.reshape(1,6,4)
    max0 = abs(np.max(tdata[0]))

    if (max0 != 0):
        tdata[0] = tdata[0] / max0
    
    train_images.append(tdata)

train_images = np.array(train_images)
bodyOutput = np.array(bodyOutput)
bodyOutput = bodyOutput.reshape(bodyOutput.shape[0],1)
print(train_images.shape)
print(bodyOutput.shape)
print(tlabel.shape)

(206, 1, 6, 4)
(206, 1)
(206, 1)


In [61]:
train_images = np.moveaxis(train_images, [1,2,3], [3,1,2])
print(train_images.shape)

(206, 6, 4, 1)


In [62]:
shufflePoints = np.random.permutation(np.arange(0, train_images.shape[0], 1))
trainingPoints = shufflePoints[0:int(train_images.shape[0]*0.95)]
validationPoints = shufflePoints[int(train_images.shape[0]*0.95):]

x_train = train_images[trainingPoints]
x_test = train_images[validationPoints]
y_train = tlabel[trainingPoints]
y_test = tlabel[validationPoints]
bodyOutput_train = bodyOutput[trainingPoints]
bodyOutput_test = bodyOutput[validationPoints]

print(x_train.shape)
print(x_test.shape)
print(bodyOutput_train.shape)
print(bodyOutput_test.shape)
print(y_train.shape)
print(y_test.shape)

(195, 6, 4, 1)
(11, 6, 4, 1)
(195, 1)
(11, 1)
(195, 1)
(11, 1)


In [63]:
num_filters = 1
filter_size = 2
initializer = keras.initializers.RandomNormal(mean=0., stddev=1.)

sequence_input = Input(batch_shape=x_train.shape, dtype=float, name='text_input')
model = (Conv2D(filters=num_filters, kernel_size=filter_size, input_shape=(6,4,1), kernel_initializer=initializer))(sequence_input)
model = (Conv2D(filters=num_filters, kernel_size=filter_size, input_shape=(5,3), kernel_initializer=initializer))(model)
model = (Flatten())(model)
af_input = Input(batch_shape=(bodyOutput_train.shape), name='af_input')
model = Concatenate()([model, af_input])
model = (Dense(16, activation='relu', kernel_initializer=initializer))(model)
model = (Dense(8, activation='relu', kernel_initializer=initializer))(model)
model = (Dense(4, activation='relu', kernel_initializer=initializer))(model)
model = (Dense(2, activation='relu', kernel_initializer=initializer))(model)
model = (Dense(1, activation='relu', kernel_initializer=initializer))(model)
model = Model(inputs=[sequence_input, af_input], outputs=model)

In [64]:
opt = keras.optimizers.Adamax(
    learning_rate=0.01,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07
    )

model.compile(
    loss="mean_absolute_error",
    optimizer=opt
    )

In [37]:
model.fit(
    [x_train, bodyOutput_train],
    y_train,
    batch_size=195,
    epochs=2500,
    validation_data=([x_test, bodyOutput_test], y_test)
    )

Train on 195 samples, validate on 11 samples
Epoch 1/2500
195/195 [==============================] - 0s 1ms/step - loss: 41.9363 - val_loss: 2.9405
Epoch 2/2500
195/195 [==============================] - 0s 95us/step - loss: 41.7634 - val_loss: 2.9299
Epoch 3/2500
195/195 [==============================] - 0s 0us/step - loss: 41.5585 - val_loss: 2.9180
Epoch 4/2500
195/195 [==============================] - 0s 80us/step - loss: 41.3553 - val_loss: 2.9058
Epoch 5/2500
195/195 [==============================] - 0s 80us/step - loss: 41.1613 - val_loss: 2.8937
Epoch 6/2500
195/195 [==============================] - 0s 0us/step - loss: 40.9750 - val_loss: 2.8840
Epoch 7/2500
195/195 [==============================] - 0s 80us/step - loss: 40.7969 - val_loss: 2.8754
Epoch 8/2500
195/195 [==============================] - 0s 0us/step - loss: 40.6217 - val_loss: 2.8665
Epoch 9/2500
195/195 [==============================] - 0s 80us/step - loss: 40.4424 - val_loss: 2.8575
Epoch 10/2500
195/195 [

In [38]:
model.save_weights('weight.h5')

In [65]:
model.load_weights('weight.h5')

In [66]:
weights = np.array(model.get_weights())
print(weights)

[array([[[[-0.5114122 ]],

        [[ 1.1474223 ]]],


       [[[ 1.2159991 ]],

        [[ 0.34990728]]]], dtype=float32)
 array([0.26477632], dtype=float32)
 array([[[[-0.65016925]],

        [[ 0.5501548 ]]],


       [[[-0.48959467]],

        [[ 1.9322684 ]]]], dtype=float32)
 array([0.3641441], dtype=float32)
 array([[-0.77956414,  0.8478649 ,  0.19543278,  0.24607895,  0.73526317,
         0.42324623,  0.23417532, -0.5125173 , -0.6806629 , -1.3779557 ,
         0.32634926,  1.8095369 ,  0.45830175, -0.3214026 , -1.255905  ,
         0.83908534],
       [ 0.12766139, -1.9304456 , -0.39179844, -1.0655394 ,  0.01800399,
        -0.4668302 , -0.9151849 ,  0.6191089 ,  0.9581525 ,  1.2446494 ,
         0.49381545, -0.7906137 ,  1.243025  , -0.39243227, -0.37591994,
         0.5704944 ],
       [-0.42515483, -2.599615  , -0.9280036 ,  1.2903694 , -0.05852915,
        -1.46907   , -0.1479898 , -1.3971093 , -1.771771  , -0.09277151,
         0.28641814, -0.95787185,  0.0462773 ,  2.6869

In [41]:
weight_file = open("weightOutput.txt", "w")

for w in weights[0:]:
    weight_file.write("{")

    if(len(w.shape)==2):
        for i in np.array(w):
            weight_file.write("{")
            i=np.array(i)

            for j in np.array(i):
                weight_file.write(str(j) + ", ")
            
            weight_file.write("}, \n")
    
    else:
        for j in np.array(w):
            weight_file.write(str(j) + ", ")
    
    weight_file.write("}\n\n")

weight_file.flush()
weight_file.close()

In [67]:
testInput = np.array([[[15.0,90.0,11.0,-1.0],[41.0,23.0,22.0,2.0],[27.0,57.0,41.0,0.0],
[22.0,80.0,30.0,0.0],[28.0,12.0,9.0,-1.0],[38.0,8.0,3.0,0.0]]]).reshape(1,1,6,4)

In [68]:
max0 = abs(np.max(testInput[0,0]))

if (max0 != 0):
    testInput[0,0] = testInput[0,0] / max0

In [69]:
testInput = np.moveaxis(testInput, [1,2,3], [3,1,2])
testBodyInput = np.array([1.]).reshape(1,1)
model.predict([testInput, testBodyInput])

array([[43.28696]], dtype=float32)

In [70]:
outputs = [K.function([model.input], [layer.output])([[testInput, testBodyInput], 1]) for layer in model.layers]
print(outputs)

[[array([[[[ 0.16666667],
         [ 1.        ],
         [ 0.12222222],
         [-0.01111111]],

        [[ 0.45555556],
         [ 0.25555557],
         [ 0.24444444],
         [ 0.02222222]],

        [[ 0.3       ],
         [ 0.6333333 ],
         [ 0.45555556],
         [ 0.        ]],

        [[ 0.24444444],
         [ 0.8888889 ],
         [ 0.33333334],
         [ 0.        ]],

        [[ 0.31111112],
         [ 0.13333334],
         [ 0.1       ],
         [-0.01111111]],

        [[ 0.42222223],
         [ 0.08888889],
         [ 0.03333334],
         [ 0.        ]]]], dtype=float32)], [array([[[[1.9703392 ],
         [0.28989285],
         [0.49454117]],

        [[0.9114375 ],
         [1.3440981 ],
         [0.7192179 ]],

        [[1.4463264 ],
         [1.6611204 ],
         [0.43713272]],

        [[1.5846606 ],
         [0.38978577],
         [0.21201763]],

        [[0.8031846 ],
         [0.43108264],
         [0.24141927]]]], dtype=float32)], [array([[[[ 1.3934